In [1]:
# !pip install jours-feries-france -q
# !pip install vacances-scolaires-france -q

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import xgboost
import category_encoders as ce
import lightgbm

import warnings
warnings.filterwarnings("ignore")

TARGETS = ["Available", "Charging", "Passive", "Other"]

In [3]:
import os
def mkdir(d):
    if not os.path.exists(d):
        os.makedirs(d)

In [4]:
def plot_feat_importance(clf, X):
    feature_imp = pd.DataFrame(
        sorted(zip(clf.feature_importances_, X.columns)),
        columns=['Value','Feature']
    )

    plt.figure(figsize=(12, 4))
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
    plt.tight_layout()
    plt.show()

In [5]:
# data loading
df_data = pd.read_csv("../public_data/train.csv")
df_test = pd.read_csv("../public_data/test.csv")

df_data['date'] = pd.to_datetime(df_data['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

df_train = df_data[(df_data["date"] > "2020-05-30")]

df_train["day"] = pd.to_datetime(df_train["date"].dt.date)
df_test["day"] = pd.to_datetime(df_test["date"].dt.date)

In [6]:
def sae(y_true, y_pred):
    """Sum of Absolute errors"""
    return(sum(abs(y_true - y_pred)))

# Modeling

## Features

In [8]:
from jours_feries_france import JoursFeries

jf = list(JoursFeries.for_year(2020).values()) + list(JoursFeries.for_year(2021).values())

from vacances_scolaires_france import SchoolHolidayDates

holidays = SchoolHolidayDates()
hd = [k for k, v in holidays.holidays_for_year(2020).items() if v["vacances_zone_c"]] \
   + [k for k, v in holidays.holidays_for_year(2021).items() if v["vacances_zone_c"]]

df_train["is_jf"] = df_train["date"].dt.date.isin(jf)
df_train["is_hd"] = df_train["date"].dt.date.isin(hd)

df_test["is_jf"] = df_test["date"].dt.date.isin(jf)
df_test["is_hd"] = df_test["date"].dt.date.isin(hd)


In [9]:
# use monthly historical temperatures 
# (not very informative but helps capturing seasonality effect)

df = pd.read_csv("../paris-historical-temperature.csv", sep=";")
df["day"] = pd.to_datetime(df["observ_date"], dayfirst=True) + pd.tseries.offsets.Day(1)
df = df[df["day"] > "2020-01-01"]
df = df.set_index("day")
df = (
    df
    .reindex(pd.date_range("2020", "2022"))
    .ffill(limit=31)
    .reset_index()
    .rename(columns={"index": "day"})
)

In [10]:
# add temperatures and one hot encode area
df_train = pd.merge(
    df_train,
    df[["day", "avg_day", "avg_night"]],
    on=["day"]
).drop("Station", axis=1)

df_train = pd.concat((
    df_train.drop("area", axis=1), 
    pd.get_dummies(df_train["area"])
), axis=1)

df_test = pd.merge(
    df_test,
    df[["day", "avg_day", "avg_night"]],
    on=["day"]
).drop("Station", axis=1)

df_test = pd.concat((
    df_test.drop("area", axis=1), 
    pd.get_dummies(df_test["area"])
), axis=1)

In [11]:
df_train.head(2)

,date,Available,Charging,Passive,Other,tod,dow,trend,Latitude,Longitude,Postcode,day,is_jf,is_hd,avg_day,avg_night,east,north,south,west
0,2020-07-03,3,0,0,0,0,6,27,48.855667,2.354089,75004,2020-07-03,False,False,26.5,15.7,0,0,1,0
1,2020-07-03,2,1,0,0,0,6,27,48.864240,2.397724,75020,2020-07-03,False,False,26.5,15.7,1,0,0,0


In [12]:
df_test.head(2)

,date,tod,dow,trend,Latitude,Longitude,Postcode,day,is_jf,is_hd,avg_day,avg_night,east,north,south,west
0,2021-02-19,0,6,22203,48.85567,2.354089,75004,2021-02-19,False,True,10.2,4.2,0,0,1,0
1,2021-02-19,0,6,22203,48.86424,2.397724,75020,2021-02-19,False,True,10.2,4.2,1,0,0,0


In [13]:
# training loop

y_preds = list()
models = list()

train_feats = df_train.drop(TARGETS + ["date", "day", "trend"], axis=1)
test_feats = df_test.drop(["date", "day", "trend"], axis=1)

for target in tqdm(TARGETS):

    train_target = df_train[target]

    lgbm = lightgbm.LGBMRegressor(verbose=0)
    lgbm.fit(
        train_feats,
        train_target,
        eval_set=[
            (train_feats, train_target), 
        ],
        eval_metric="l1",
        verbose=50,
        categorical_feature=["Postcode"]
    )

    y_preds.append(lgbm.predict(test_feats))
    models.append(lgbm)
    
    # plot_feat_importance(lgbm, train_feats)

  0%|          | 0/4 [00:00<?, ?it/s]

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015266 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[50]	training's l1: 0.726596	training's l2: 0.800949
[100]	training's l1: 0.696127	training's l2: 0.756391
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016859 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[50]	training's l1: 0.328624	training's l2: 0.22229
[100]	training's l1: 0.318463	training's l2: 0.215537
[LightGBM] [Warning] Met categorical feature which co

In [14]:
preds = np.vstack(y_preds).T
preds.shape

(165984, 4)

In [15]:
test_station = pd.read_csv("../public_data/test.csv")

station_prediction = pd.concat(
    (test_station, pd.DataFrame(preds, columns=TARGETS)),
    axis=1
)[["date", "area", "Station"] + TARGETS]


In [16]:
# normalize in a simple but efficient way
for target in TARGETS:
    station_prediction[target] = 3 * station_prediction[target] / station_prediction.sum(axis=1)

In [17]:
mkdir("sample_result_submission")

station_prediction[["date", "area", "Station"] + TARGETS].round().to_csv(
    "../output/sample_result_submission/station.csv", index=False
)

In [18]:
station_prediction.round().head()

,date,area,Station,Available,Charging,Passive,Other
0,2021-02-19 00:00:00,south,FR*V75*EBELI*1*1,3.0,0.0,0.0,0.0
1,2021-02-19 00:00:00,east,FR*V75*EBELI*10*1,1.0,0.0,0.0,2.0
2,2021-02-19 00:00:00,west,FR*V75*EBELI*11*1,1.0,0.0,0.0,2.0
3,2021-02-19 00:00:00,south,FR*V75*EBELI*12*1,3.0,0.0,0.0,-0.0
4,2021-02-19 00:00:00,north,FR*V75*EBELI*13*1,2.0,0.0,0.0,0.0


# Area Level

In [19]:
# aggregate station prediction at the area level
area_prediction = station_prediction.groupby(["date", "area"])[TARGETS].sum().reset_index()
area_prediction.to_csv(
    "../output/sample_result_submission/area.csv", index=False
)

In [20]:
area_prediction.head()

,date,area,Available,Charging,Passive,Other
0,2021-02-19 00:00:00,east,38.153711,6.565524,2.815029,27.713173
1,2021-02-19 00:00:00,north,36.224971,4.311193,4.852075,20.123767
2,2021-02-19 00:00:00,south,41.298428,5.165521,3.436689,12.784591
3,2021-02-19 00:00:00,west,27.642690,8.372816,6.592692,26.949573
4,2021-02-19 00:15:00,east,38.153711,6.565524,2.815029,27.713173


# Global Level

In [21]:
# aggregate station prediction at the area level
global_prediction = station_prediction.groupby(["date"])[TARGETS].sum().reset_index()
global_prediction.to_csv(
    "../output/sample_result_submission/global.csv", index=False
)

In [22]:
global_prediction.head()

,date,Available,Charging,Passive,Other
0,2021-02-19 00:00:00,143.319800,24.415055,17.696486,87.571104
1,2021-02-19 00:15:00,143.319800,24.415055,17.696486,87.571104
2,2021-02-19 00:30:00,143.386248,24.294006,17.699792,87.596382
3,2021-02-19 00:45:00,143.612550,23.896161,17.709829,87.671880
4,2021-02-19 01:00:00,143.675892,23.520718,18.008982,87.672604


In [23]:
shutil.make_archive("../output/sample_result_submission", "zip", "sample_result_submission")

'/Users/paulemiledugnat/Desktop/codalab/notebooks/sample_result_submission.zip'